In [ ]:
import json
import pandas as pd
import ast
from collections import defaultdict
import os
from postprocess import ResultProcess, Eval
import pandas as pd 
from const import Const as C
import re

In [ ]:
dataset_name="News" #"Movie" #"Book"  Music Movie  News
result_folder="llama_result"
base_folder=f"./{result_folder}/{dataset_name}"
from get_transition_mat_70b import get_transition_mat
folder_path=f"{base_folder}/{dataset_name}_train"
num_options = 5 # replace with the number of options
trans_mat = get_transition_mat(folder_path, num_options)


In [ ]:
def show_in_mat_sty(transitions):
    import numpy as np
    # Order of keys
    keys = sorted(transitions.keys())
    # Create 5x5 matrix
    matrix = np.zeros((5, 5))
    for i, key1 in enumerate(keys):
        for j, key2 in enumerate(keys):
            matrix[i, j] = transitions[key1][key2]

    return matrix

total_samples = sum([sum(value.values()) for value in trans_mat.values()])
each_train_num={k:sum(trans_mat[k].values()) for k in trans_mat.keys()}
prior_prob = {k: sum(value.values())/ total_samples for value in trans_mat.values() for k in ['A', 'B', 'C', 'D', 'E']}
likelihood_prob = {k: {evi:trans_mat[k][evi]/each_train_num[k] for evi in trans_mat[k].keys() } for k in ['A', 'B', 'C', 'D', 'E']}


In [ ]:
total_samples = sum([sum(value.values()) for value in trans_mat.values()])
each_train_num={k:sum(trans_mat[k].values()) for k in trans_mat.keys()}
prior_prob = {k: sum(value.values())/ total_samples for value in trans_mat.values() for k in ['A', 'B', 'C', 'D', 'E']}
likelihood_prob = {k: {evi:trans_mat[k][evi]/each_train_num[k] for evi in trans_mat[k].keys() } for k in ['A', 'B', 'C', 'D', 'E']}


In [ ]:
import pandas as pd
# Function to load jsonl file into a pandas DataFrame
def load_jsonl_to_dataframe(file_path):
    with open(file_path, 'r') as file:
        # Parse each line as a JSON object and collect into a list
        jsonl_content = [json.loads(line) for line in file]
    # Convert list of JSON objects to a pandas DataFrame
    return pd.DataFrame(jsonl_content)

In [ ]:

def cal_post(evi,prior_prob):
    posterior_prob={}

    for hpy in trans_mat.keys():
        posterior_prob[hpy]=prior_prob[hpy]*likelihood_prob[hpy][evi]
    posterior_prob={key:posterior_prob[key]/sum(posterior_prob.values()) for key in posterior_prob.keys()}
    return posterior_prob

def cal_s(file_path,content_s,content_post,content,ranked_origin_pos,post_ranked_pos,df):
    valid_num=0
    except_num=0    
    with open(file_path, 'r') as file:
        for line in file:
            data = json.loads(line)
            # rank_order = data['response']['gpt_data']['choices'][0]['message']['content']
            try:
                text = data['response']#['result']#['choices'][0]['message']['content']#['gpt_data']
            except:
                except_num+=1
                continue
            match = re.search(r'"rank_order":\s*"([^"]*)"', text, re.DOTALL)
            
            if not match:
                except_num+=1
                continue
            else:
                order = match.group(1)
                task_id=data["task_id"]
                request_data=df[df['task_id'] == task_id]
                origin_pos_line=request_data["original_positions_after_shuffle"].values[0]
                order_number = []
                for item in order.split(' '):
                    if len(item) == 1 and item in ['A','B','C','D','E']:
                        order_number.append(ord(item) - ord('A'))
                    else:
                        continue
                        print(f"item:{item}")
                if len(order_number)>len(origin_pos_line):
                    continue
                ranked_origin_pos_line=[]
                for i in order_number:
                    if i<len(origin_pos_line):
                        # print(i,'--',len(origin_pos_line))
                        ranked_origin_pos_line.append(origin_pos_line[i])
                    else:
                        print("i outOfindex:",i)
                # ranked_origin_pos_line=[origin_pos_line[i] for i in order_number]
                if len(ranked_origin_pos_line)==0:
                    continue
                if task_id not in content_s.keys():
                    content_s[task_id]=[]
                    content_post[task_id]=[{'A':0.2,'B':0.2,'C':0.2,'D':0.2,'E':0.2}] 
                    content[task_id]=[]
                    ranked_origin_pos[task_id]=[]

                    post_ranked_pos[task_id]=[]
                    # content_s[task_id].append(sum(-p*math.log(p) for p in content_post[task_id][-1].values()))
# 
                content_post[task_id].append(cal_post(chr(ord('A')+ranked_origin_pos_line[0]),content_post[task_id][-1]))
                content_s[task_id].append(sum(-p*math.log(p) if p > 0 else 0 for p in content_post[task_id][-1].values()))
                # sum(-p*math.log(p) if p > 0 else 0 for p in content_post[task_id][-1].values())

                content[task_id].append(order)
                ranked_origin_pos[task_id].append(ranked_origin_pos_line)
                
                post_p=content_post[task_id][-1]
                sorted_data = sorted(post_p.items(), key=lambda x: x[1], reverse=True)
                post_ranked_pos[task_id].append([ord(item[0])-ord('A') for item in sorted_data])
                valid_num+=1
    print(f"valid_num:{valid_num},except_num:{except_num}")
    return content_s,content_post,content,ranked_origin_pos,post_ranked_pos

In [ ]:

def get_by_e(content_s,ranked_origin_pos,content_post,num=10,save_multi=True):
    final_to_conbined={}
    def tend_decrease(input:list):
        if input[-1]<input[-2] and input[-2]<input[-3]:
            return True
        else:
            return False
    no_shoulian=0
    extra_cnt=0
    record={}
    used_e={}
    for i in range(3,num):
        record[i]=0

    for key,val in content_s.items():
        key_ranked_pos=ranked_origin_pos[key]
        key_num=int(key.split("-")[0])
        shoulian=False
        for i in range(3,len(val)):
            if not tend_decrease(val[i-3:i]):
                if not shoulian:
                    if not save_multi:
                        final_to_conbined[key_num]=key_ranked_pos[i]
                    else:
                        final_to_conbined[key_num]=key_ranked_pos[i-3:i]

                else:
                    extra_cnt+=1

                    post_p=content_post[key][i]
                    sorted_data = sorted(post_p.items(), key=lambda x: x[1], reverse=True)
                    if save_multi:
                        order=[ord(item[0])-ord('A') for item in sorted_data]
                        final_to_conbined[key_num]=[order]
                    else:
                        final_to_conbined[key_num]=[ord(item[0])-ord('A') for item in sorted_data]
                    # print(post_p)
                    
                record[i]+=1
                used_e[key]=i
                break
            else:
                if i==len(val)-1:
                    if not save_multi:
                        final_to_conbined[key_num]=key_ranked_pos[i]
                    else:
                        final_to_conbined[key_num]=key_ranked_pos[i-3:i]

                    used_e[key]=i

                    no_shoulian+=1
                    # print(f"{key} no shoulian {content_s[key]}")
                    record[i]+=1
                    break
                else:
                    i+=1
                    shoulian=True
                    continue
    print("no_shoulian num:",no_shoulian,"in first 3:",extra_cnt)
    print("get value position counts:",record)
    return final_to_conbined,used_e
# final_to_conbined=get_by_e(content_s,ranked_origin_pos,content_post,num=10,save_multi=save_multi)#
# final_to_conbined

In [ ]:

def get_y_true(data: pd.DataFrame, begin_index: int, end_index: int):
    """Get ground truth from data.
    Args:
        data (pd.DataFrame): The Dataframe of data.
        begin_index (int): The begin index of data.
        end_index (int): The end index of data.

    Returns:
        Dict[int, List[int]]: The ground truth from begin index to end index.
    """
    y_true = data[begin_index:end_index].pos_target_index.to_dict()

    return y_true

# y_pred={}
def conbine_by_score(to_conbined):
    def score_cal(scores,pred_list):
        # scores=[0,0,0,0,0]
        # print(scores)
        for i in range(len(pred_list)):
            # print(scores[pred_list[i]],len(pred_list))
            scores[pred_list[i]] += len(pred_list) - i
        return scores
    y_pred={}
    for key,val in to_conbined.items():
        scores=[0,0,0,0,0]
        for one_pred in val:
            # print(one_pred)
            scores=score_cal(scores,one_pred)
        
        idx_score={}
        for idx,score0 in enumerate(scores):
            idx_score[idx]=score0

        sorted_data = sorted(idx_score.items(), key=lambda x: x[1], reverse=True)

        sorted_idx = [item[0] for item in sorted_data]
        y_pred[key]= sorted_idx

    return y_pred

def combine_by_voting(to_combine):
    def most_common_in_position(rankings, position):
        # Count occurrences of each element at a given position
        elements_at_position = [ranking[position] for ranking in rankings if position<len(ranking)]
        element_counts = {}
        for element in elements_at_position:
            if element in element_counts:
                element_counts[element] += 1
            else:
                element_counts[element] = 1
        # Get the most common element at the given position
        most_common_element = max(element_counts, key=element_counts.get)
        return most_common_element
    aggregated_rankings = {}
    for key, rankings in to_combine.items():
        if len(rankings) == 1:
            aggregated = rankings
        else:
            aggregated = [most_common_in_position(rankings, i) for i in range(len(rankings[0]))]

        aggregated_rankings[key] = aggregated

    return aggregated_rankings

def get_from_file(response_path,df):
    result = {}
    valid_num=0
    except_num=0
    with open(response_path, 'r') as file:
        for line in file:
            data = json.loads(line)
            # rank_order = data['response']['gpt_data']['choices'][0]['message']['content']
            try:
                text = data['response']#['result']#['choices'][0]['message']['content']#['gpt_data']
            except:
                except_num+=1
                continue
            # match = re.search(r'\"rank_order\": \"(.*?)\"\\n', text)
            match = re.search(r'"rank_order":\s*"([^"]*)"', text, re.DOTALL)
            if not match:
                except_num+=1
                continue
            else:
                order = match.group(1)
                # print('rank_order',rank_order)
                
                # print('text:',text,'match:',match)
                task_id=data["task_id"]
                request_data=df[df['task_id'] == task_id]
                origin_pos_line=request_data["original_positions_after_shuffle"].values[0]
                order_number = []
                for item in order.split(' '):
                    if len(item) == 1 and item in ['A','B','C','D','E']:
                        order_number.append(ord(item) - ord('A'))
                    else:
                        continue
                        print(f"item:{item}")
                if len(order_number)>len(origin_pos_line):
                    continue
                ranked_origin_pos_line=[]
                for i in order_number:
                    if i<len(origin_pos_line):
                        # print(i,'--',len(origin_pos_line))
                        ranked_origin_pos_line.append(origin_pos_line[i])
                    else:
                        print("i outOfindex:",i)
                group_id = int(task_id.split("-")[0])
                result[group_id] = ranked_origin_pos_line
                valid_num+=1
        print(f"valid_num:{valid_num},except_num:{except_num}")
        return result
def get_from_multi_files(exps):
    # exps=[0,1,2]
    all_y_preds={}
    for exp in exps:
        df = load_jsonl_to_dataframe(f"{base_folder}/{dataset_name}_test/{exp}/gpt-3.5-turbo/idx{begin_index}-{end_index}_posIdx@-1_shuffle_neg_cands@1_example@1_list_candidate@5_history@5/request.jsonl")

        response_path=f"{data_folder}/{dataset_name}_test/{exp}/gpt-3.5-turbo/idx{begin_index}-{end_index}_posIdx@-1_shuffle_neg_cands@1_example@1_list_candidate@5_history@5/response.jsonl"
        y_pred=get_from_file(response_path,df)
        for k,v in y_pred.items():
            if k not in all_y_preds.keys():
                all_y_preds[k]=[v]
            else:
                all_y_preds[k].append(v)
    return all_y_preds
    # conbined_y_pred={}

def get_each_file_result(data_folder,exps,begin_index,end_index):
    ndcg1s = []
    for exp in exps:
        file_path=f"{base_folder}/{dataset_name}_test/{exp}/gpt-3.5-turbo/idx{begin_index}-{end_index}_posIdx@-1_shuffle_neg_cands@1_example@1_list_candidate@5_history@5/result.jsonl"
        with open(file_path, 'r') as file:
            for line in file:
                data = json.loads(line)
                if "NDCG@1" in data:
                    ndcg1 = data["NDCG@1"]
                    ndcg1s.append(ndcg1)
                    break
    return sum(ndcg1s)/len(ndcg1s),ndcg1s

def get_by_min_entropy(content_s, ranked_origin_pos, num=10, save_multi=True):
    final_to_combined = {}
    used_e = {}
    cnt={}
    for i in range(num):
        cnt[i]=0
    # print('ranked_origin_pos:',ranked_origin_pos)
    for key, val in content_s.items():
        key_ranked_pos = ranked_origin_pos[key]
        key_num = int(key.split("-")[0])
        
        # Find the position of minimum entropy
        min_entropy_pos = val.index(min(val))
        cnt[min_entropy_pos]+=1
        # Save the corresponding model output / posterior probability ranking result
        if save_multi:
            # Save multiple results
            final_to_combined[key_num] = key_ranked_pos[max(0, min_entropy_pos-3) : min_entropy_pos+1]
        else:
            # Save one result
            final_to_combined[key_num] = key_ranked_pos[min_entropy_pos]

        used_e[key] = min_entropy_pos
    print('min_entropy_index_cnt:',cnt)
    return final_to_combined, used_e
def copeland_aggregation(to_combined):
    """
    Aggregates multiple ranking lists using the Copeland's method.

    :param rankings_list: List of rankings.
    :return: Aggregated ranking based on Copeland's method.
    """
    result={}
    for key, rankings_list in to_combined.items():
        num_items = len(rankings_list[0])  # Get the number of items to rank.
        scores = [0] * num_items

        # Calculate Copeland scores
        for rankings in rankings_list:
            for i in range(num_items):
                for j in range(i + 1, num_items):
                    if rankings.index(i) < rankings.index(j):
                        scores[i] += 1
                    else:
                        scores[j] += 1

        # Sort items by Copeland scores
        sorted_items = sorted(range(num_items), key=lambda k: -scores[k])
        # print(sorted_items,)
        result[key] = sorted_items
    return result
def median_aggregation(to_combined):
    """
    Aggregates multiple ranking lists using the Median method.

    :param to_combined: Dictionary where each key has a list of rankings.
    :return: A dictionary of aggregated rankings.
    """
    result = {}
    
    for key, rankings_list in to_combined.items():
        num_items = len(rankings_list[0])  # Get the number of items to rank.
        median_ranks = {}

        # Calculate median rank for each item
        for i in range(num_items):
            ranks_of_i = [rankings.index(i) for rankings in rankings_list]
            sorted_ranks = sorted(ranks_of_i)

            # If there's an even number of rankings, take the average of the two middle ranks
            if len(sorted_ranks) % 2 == 0:
                median_rank = (sorted_ranks[len(sorted_ranks) // 2 - 1] + sorted_ranks[len(sorted_ranks) // 2]) / 2
            else:
                median_rank = sorted_ranks[len(sorted_ranks) // 2]

            median_ranks[i] = median_rank

        # Sort items by their median rank
        sorted_items = sorted(median_ranks.keys(), key=lambda k: median_ranks[k])
        result[key] = sorted_items
    return result
content_s={}
content_post={}
content_order={}
ranked_origin_pos={}
post_ranked_pos={}

num=10
all_numbers = list(range(20))
start_idx=0
exps_cands=range(start_idx,start_idx+num) # [0,1,2]

exps=[2,3,4]

data_folder=base_folder
save_multi=True

data=pd.read_csv(f"./data/{dataset_name}/LLM/topk_candidate@5_history@5_origin.csv",delimiter="\t")#, usecols=C.DATA_COL_NAME_LIST
y_true=get_y_true(data,begin_index,end_index)
for k,v in y_true.items():
    y_true[k]=[v]

for exp in exps_cands:
    df = load_jsonl_to_dataframe(f"{base_folder}/{dataset_name}_test/{exp}/gpt-3.5-turbo/idx{begin_index}-{end_index}_posIdx@-1_shuffle_neg_cands@1_example@1_list_candidate@5_history@5/request.jsonl")
    
    file_path = f'{base_folder}/{dataset_name}_test/{exp}/gpt-3.5-turbo/idx{begin_index}-{end_index}_posIdx@-1_shuffle_neg_cands@1_example@1_list_candidate@5_history@5/response.jsonl'
    content_s,content_post,content_order,ranked_origin_pos,post_ranked_pos=cal_s(file_path,content_s,content_post,content_order,ranked_origin_pos,post_ranked_pos,df)  
booststrap_y=get_from_multi_files(exps)

result_path=f"{base_folder}/result012.jsonl"
candidate_num=5

methods =['entropy','score']# ,'score','single_file' [  'entropy',]#'voting', ,'other'
# methods=['other','entropy']
results = {}

for method in methods:
    if method == 'score':
        y_pred = conbine_by_score(booststrap_y)
    elif method == 'other':
        # y_pred = combine_by_voting(booststrap_y)
        y_pred=copeland_aggregation(booststrap_y)
        
        # y_pred=median_aggregation(booststrap_y)
    elif method == 'single_file':
        avg,each_res=get_each_file_result(data_folder,exps,begin_index,end_index)
        #
        # y_pred = get_from_file(seleted_file_path)
        results[f"avg_{exps}"]=round(avg,4)
        results[f"min_{exps}"]=round(min(each_res),4)
        # results[f"avg_{exps}"]=avg

        results[f"max_{exps}"]=round(max(each_res),4)
        continue
    elif method == 'entropy':
        # final_to_conbined,used_e=get_by_e(content_s,ranked_origin_pos,content_post,num=num,save_multi=save_multi)#
        final_to_conbined,used_e=get_by_min_entropy(content_s,ranked_origin_pos,num=num,save_multi=save_multi)#
        # print(final_to_conbined)
        if save_multi:
            y_pred = conbine_by_score(final_to_conbined)
        else:
            y_pred = final_to_conbined
    else:
        continue
    result = Eval.eval_result(
        task="List",
        result_path=result_path,
        y_true=y_true,
        y_pred=y_pred,
        topk=[1,2,3,4,5][:5], # or [1,2,3,4,5,6,7,8,10] based on your needs
        candidate_num=candidate_num
    )
    results[method] = result[3]["Precision@1"]
print(f"{dataset_name} exp {num}:\n",results)

In [ ]:
import pandas as pd
import random
methods = ['score','voting','unweighted_entropy','entropy','sc-entropy'] #
total_num=20
def load_data(file_path):
    return pd.read_csv(file_path, delimiter="\t")

def calculate_metrics(begin_index, end_index, num, start_idx, base_folder, dataset_name):
    # 初始化字典
    content_s = {}
    content_post = {}
    content_order = {}
    ranked_origin_pos = {}
    post_ranked_pos = {}

    
    exps_cands = range(start_idx, start_idx + num)
    exps_cands = list(exps_cands)
    # print(f"exps_cands: {exps_cands}")
    for i, exp in enumerate(exps_cands):
        exps_cands[i] = exp%total_num
    print(f"exps_cands: {exps_cands}")
    
    data_folder = dataset_name
    save_multi = False
    origin_data = f"./data/{dataset_name}/LLM/topk_candidate@5_history@5_origin.csv"
    data = pd.read_csv(origin_data, delimiter="\t")

    y_true = get_y_true(data, begin_index, end_index)
    for k, v in y_true.items():
        y_true[k] = [v]

    for exp in exps_cands:
        file_path = f'{base_folder}/{data_folder}_test/{exp}/gpt-3.5-turbo/idx{begin_index}-{end_index}_posIdx@-1_shuffle_neg_cands@1_example@1_list_candidate@5_history@5/response.jsonl'
        df = load_jsonl_to_dataframe(f"{base_folder}/{dataset_name}_test/{exp}/gpt-3.5-turbo/idx{begin_index}-{end_index}_posIdx@-1_shuffle_neg_cands@1_example@1_list_candidate@5_history@5/request.jsonl")

        content_s, content_post, content_order, ranked_origin_pos, post_ranked_pos = cal_s(file_path, content_s, content_post, content_order, ranked_origin_pos, post_ranked_pos,df)

    booststrap_y = get_from_multi_files(exps_cands)
    # print(f"booststrap_y: {booststrap_y}")
    result_path = f"{base_folder}/result012.jsonl"
    candidate_num = 5
    results = {}
    pre_correct={}

    for method in methods:
        print(f"method: {method}")
        if method == 'entropy':
            final_to_conbined, used_e = get_by_min_entropy(content_s, ranked_origin_pos, num=num, save_multi=True)
            y_pred = conbine_by_score(final_to_conbined)
        if method == 'sc-entropy':
            final_to_conbined, used_e = get_by_min_entropy(content_s, ranked_origin_pos, num=num, save_multi=True)
            y_pred = combine_by_voting(final_to_conbined)
        elif method == 'unweighted_entropy':
            final_to_conbined, used_e = get_by_min_entropy(content_s, ranked_origin_pos, num=num, save_multi=False)
            y_pred = final_to_conbined
        elif method == 'score':
            y_pred = conbine_by_score(booststrap_y)
        # print('y_pred:\n', y_pred, '\ny_true:\n', y_true)
        elif method == 'voting':
            y_pred = combine_by_voting(booststrap_y)
        result = Eval.eval_result(
            task="List",
            result_path=result_path,
            y_true=y_true,
            y_pred=y_pred,
            topk=[1,2,3,4,5][:5],  # or [1,2,3,4,5,6,7,8,10] based on your needs
            candidate_num=candidate_num
        )
        print(f"result: {result}")
        results[method] = result[3]["Precision@1"]
        pre_correct[method]={}
        for k,v in y_pred.items():
            if v[0]==y_true[k][0]:
                pre_correct[method][k]=1
            else:
                pre_correct[method][k]=0
    print(f"results: {results}")
    print('='*20)
    return results,pre_correct

def process_for_start_idx(begin_index, end_index, num, start_idx, base_folder, dataset_name):

    results,pre_correct = calculate_metrics(begin_index, end_index, num, start_idx, base_folder, dataset_name)
    return results,pre_correct


print(dataset_name)
start_idx_values = [0,3,6]#[9,4,7]#range(9,total_num,3)  # start_idx 的范围 1
print(f"start_idx_values: {list(start_idx_values)}")
all_results = {}
pre_corrects={}

for start_idx in start_idx_values:
    results,pre_correct = process_for_start_idx(begin_index, end_index, num, start_idx, base_folder, dataset_name)
    all_results[start_idx] = results 
    pre_corrects[start_idx]=pre_correct

